# Exercise 6.15

In [1]:
import pathlib
import subprocess
import numpy as np
import pandas as pd
from IPython.display import display, Math
from scipy import linalg as la
from scipy import stats
import TwoWayManova
from TwoWayManovaEffectTests import TwoWayManovaEffectTests

In [2]:
def load_data() -> pd.DataFrame:
    fpth = pathlib.Path(r'..\..\data\Table6.4.xlsx')
    df = pd.read_excel(fpth)
    df.rename(columns={'rate of extrusion': 'factor1',
                       'amount of additive': 'factor2',
                       'tear resistance': 'x1',
                       'gloss': 'x2',
                       'opacity': 'x3'}, inplace=True)
    
    df['factor1'] = df['factor1'].astype('category')
    df['factor2'] = df['factor2'].astype('category')

    return df

In [3]:
film_df = load_data()

- $X_{1} = $ tear resistance
- $X_{2} = $ gloss
- $X_{3} = $ opacity

## (a)

In [4]:
fit = TwoWayManova.TwoWayManova(film_df, factor1_col='factor1', factor2_col='factor2', var_cols=['x1','x2','x3'])

In [5]:
fit.display_manova_table()

<IPython.core.display.Math object>

In [6]:
B1 = fit.manova.B1
B2 = fit.manova.B2
W = fit.manova.W

g = fit._design.metadata.g
b = fit._design.metadata.b
n = fit._design.metadata.n
p = fit._design.metadata.p

alpha = 0.05

In [7]:
lambda_star_1 = la.det(W)/la.det(B1 + W)
const_1 = -(g*b*(n-1) - (p + 1 - (g - 1))/2)
test_stat_1 = const_1 * np.log(lambda_star_1)

In [8]:
display(Math(r'X_{1}^{2\star}'
             '='
             r'- \left[ gb(n-1) - \frac{p + 1 - (g-1)}{2} \right]'
             r'\ln \Lambda_{1}^{\star}'
             '='
             f'{test_stat_1:.2f}'
             ))

<IPython.core.display.Math object>

In [9]:
chi2_crit_1 = stats.chi2.ppf(1-alpha, (g-1)*p)
display(Math(r'\chi_{(g-1)p}^{2}(\alpha)'
             '='
             fr'\chi_{{ ({g - 1}){p} }}({alpha})'
             '='
             f'{chi2_crit_1:.2f}'
             ))

<IPython.core.display.Math object>

In [10]:
if test_stat_1 > chi2_crit_1:
    display(Math(fr'\text{{We have that }} X_{{1}}^{{2\star}} = {test_stat_1:.3f} > '
                 r'X_{\text{crit}}^{2}'
                 '='
                 fr'\chi^{{2}}_{{{p*(g-1)}}} \left( {alpha} \right) = {chi2_crit_1:.3f}'
                 r'\text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\tau}_{1} = \bm{\tau}_{2} = \textbf{0}'
                 ))
else:
    display(Math(fr'\text{{We have that }} X_{{1}}^{{2\star}} = {test_stat_1:.3f} \leq '
                 r'X_{\text{crit}}^{2}'
                 '='
                 fr'\chi^{{2}}_{{{p*(g-1)}}} \left( {alpha} \right) = {chi2_crit_1:.3f}'
                 r'\text{{, so we would fail to reject the null hypothesis that }} '
                 r'\bm{\tau}_{1} = \bm{\tau}_{2} = \textbf{0}'
                 ))

<IPython.core.display.Math object>

In [11]:
lambda_star_2 = la.det(W)/la.det(B2 + W)
const_2 = -(g*b*(n-1) - (p + 1 - (b - 1))/2)
test_stat_2 = const_2 * np.log(lambda_star_2)

In [12]:
display(Math(r'X_{2}^{2\star}'
             '='
             r'- \left[ gb(n-1) - \frac{p + 1 - (b-1)}{2} \right]'
             r'\ln \Lambda_{2}^{\star}'
             '='
             f'{test_stat_2:.2f}'
             ))

<IPython.core.display.Math object>

In [13]:
chi2_crit_2 = stats.chi2.ppf(1-alpha, (b-1)*p)
display(Math(r'\chi_{(b-1)p}^{2}(\alpha)'
             '='
             fr'\chi_{{ {b - 1}{p} }}({alpha})'
             '='
             f'{chi2_crit_2:.2f}'
             ))

<IPython.core.display.Math object>

In [14]:
if test_stat_2 > chi2_crit_2:
    display(Math(fr'\text{{We have that }} X_{{2}}^{{2\star}} = {test_stat_2:.3f} > '
                 r'X_{\text{crit}}^{2}'
                 '='
                 fr'\chi^{{2}}_{{{p*(b-1)}}} \left( {alpha} \right) = {chi2_crit_2:.3f}'
                 r'\text{{, so we would reject the null hypothesis that }} '
                 r'\bm{\beta}_{1} = \bm{\beta}_{2} = \textbf{0}'
                 ))
else:
    display(Math(fr'\text{{We have that }} X_{{2}}^{{2\star}} = {test_stat_2:.3f} \leq '
                 r'X_{\text{crit}}^{2}'
                 '='
                 fr'\chi^{{2}}_{{{p*(b-1)}}} \left( {alpha} \right) = {chi2_crit_2:.3f}'
                 r'\text{{, so we would fail to reject the null hypothesis that }} '
                 r'\bm{\beta}_{1} = \bm{\beta}_{2} = \textbf{0}'
                 ))

<IPython.core.display.Math object>

The $\chi^{2}$ approximation test for both factors effects (from page 317) are significant at the 0.05-level. This is the same conclusion as the $F$-tests from Example 6.13.

## (b)

In [15]:
# Five observations for three measurements (response) for each of the two Factor 1 levels.
xbar_1 = fit.x_breakdown.means.Factor1Mean[:,:,0]

In [16]:
nu = g*b*(n-1)
diff_1 = (xbar_1[:,0] - xbar_1[:,1])[:,np.newaxis]
t_crit_1 = stats.t.ppf(1-(alpha/(p*g*(g-1))), df=nu)
std_err_1 = np.sqrt(np.diag(W)[:,np.newaxis]/nu)
scalar_1 = np.sqrt(2/(b*n))

ci_1 = diff_1 + np.array([-1, 1])*t_crit_1*std_err_1*scalar_1

In [17]:
for i in range(ci_1.shape[0]):
    display(Math(r'\begin{array}{rl}'
             fr'\tau_{{ {1} {i+1}}} - \tau_{{ {2} {i+1}}} '
             fr'\text{{ belongs to }} & \\'
             fr'& \hat{{\tau}}_{{ {1} {i+1}}} - \hat{{\tau}}_{{ {2} {i+1}}} \pm '
             f't_{{ {nu} }} ( {alpha/(p*g*(g-1)):.5f} )'
             fr'\sqrt{{ \frac{{ E_{{ {i+1}, {i+1} }} }}{{ \nu }} \frac{{ 2 }}{{ bn }} }} \\'
             fr' &  = {diff_1[i,0]:.3f} \pm {t_crit_1:.2f} ( {std_err_1[i,0]*scalar_1:.5f} ) \\'
             fr' & = {diff_1[i,0]:.3f} \pm {t_crit_1*std_err_1[i,0]*scalar_1:.3f} \text{{, or }} ({ci_1[i,0]:.3f}, {ci_1[i,1]:.3f})'
             r'\end{array}'
             ))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

We are 95% confident that the mean difference between low (-10%) and high (10%) change in rate of extrusion for tear resistance is between -0.987 and -0.193, so high extrusion has a larger mean than low extrusion.

There is an effect for change in rate of extrusion for test resistence. The high extrusion has a higher mean tear resistance than low extrusion mean resistence.

In [18]:
# Five observations for three measurements (response) for each of the two Factor 2 levels.
xbar_2 = fit.x_breakdown.means.Factor2Mean[:,0,:]

In [19]:
diff_2 = (xbar_2[:,0] - xbar_2[:,1])[:,np.newaxis]
t_crit_2 = stats.t.ppf(1-(alpha/(p*b*(b-1))), df=nu)
std_err_2 = np.sqrt(np.diag(W)[:,np.newaxis]/nu)
scalar_2 = np.sqrt(2/(g*n))

ci_2 = diff_2 + np.array([-1, 1])*t_crit_2*std_err_2*scalar_2

In [20]:
for i in range(ci_2.shape[0]):
    display(Math(r'\begin{array}{rl}'
             fr'\beta_{{ {1} {i+1}}} - \beta_{{ {2} {i+1}}} '
             fr'\text{{ belongs to }} & \\'
             fr'& \hat{{\beta}}_{{ {1} {i+1}}} - \hat{{\beta}}_{{ {2} {i+1}}} \pm '
             f't_{{ {nu} }} ( {alpha/(p*g*(g-1)):.5f} ) '
             fr'\sqrt{{ \frac{{ E_{{ {i+1}, {i+1} }} }}{{ \nu }} \frac{{ 2 }}{{ gn }} }} \\'
             fr' & = {diff_2[i,0]:.3f} \pm {t_crit_2:.2f} ( {std_err_2[i,0]*scalar_2:.5f} ) \\'
             fr' & = {diff_2[i,0]:.3f} \pm {t_crit_2*std_err_2[i,0]*scalar_2:.3f} \text{{, or }} ({ci_2[i,0]:.3f}, {ci_2[i,1]:.3f})'
             r'\end{array}'
             ))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Looks like all these intervals contain 0. This may be due to the Bonferroni adjustment we're doing here for the pairwise comparisons that make the intervals a bit wider.